In [9]:
import gradio as gr
import time
import os
import sys
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
from aip import AipSpeech
import speech_recognition as sr
import pygame
 
APP_ID = '117045037'
API_KEY = 'QTHyrEY1qfGVyHBZNRgSQXFy'
SECRET_KEY = 'Xnh8clQsR8cZu9kDiVe4cOp5qrBmQnbb'
client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)
        
def init():
    global stamp, wj_flag, item
    wj_flag = False
    stamp = time.localtime()
    item = []

init()
src = 'C:\\Users\\26073\\Workspace\\detector\\'

def synthesize_speech(text, audio_filename):
    # 调用百度TTS接口进行合成
    result = client.synthesis(text, 'zh', 1, {
        'vol': 5,  # 音量 0-15
        'per': 4,  # 选择发音人，0-普通女声，1-普通男声，3-情感合成女声，4-情感合成男声
        'spd': 5,  # 语速 0-15
        'pit': 5   # 音调 0-15
    })
    if not isinstance(result, dict):
        save_path = src + audio_filename
        with open(save_path, 'wb') as f:
            f.write(result)  # 保存音频到文件
        print(f"语音合成成功，音频保存为 {save_path}")
        play_audio(save_path)
    else:
        print("语音合成失败")

def play_audio(file_path):
    pygame.mixer.init()
    pygame.mixer.music.load(file_path)
    pygame.mixer.music.play()
    init()

def calculate(img, acc):
    global wj_flag
    img_name = "demo" + time.strftime("%Y%m%d%H%M%S", stamp)
    img.save(src + img_name + ".jpg", format="JPEG")
    prj_path = src + "_" + img_name
    
    mod_name = ""
    match acc:
        case 1:
            mod_name = "yolo11n.pt"
        case 2:
            mod_name = "yolov8n-oiv7.pt"
        case 3:
            mod_name = "yolo11s.pt"
        case 4:
            mod_name = "yolov8s-oiv7.pt"
        case 5:
            mod_name = "yolo11m.pt"
        case 6:
            mod_name = "yolov8m-oiv7.pt"
        case 7:
            mod_name = "yolo11l.pt"
        case 8:
            mod_name = "yolov8l-oiv7.pt"
        case 9:
            mod_name = "yolo11x.pt"
        case 10:
            mod_name = "yolov8x-oiv7.pt"
    model = YOLO(src + "model/" + mod_name)
    model.predict(
        source = src + img_name + ".jpg",
        save = True,
        save_txt = True,
        project = src,
        name = prj_path,
    )
    
    if os.path.exists(prj_path + "/labels/" + img_name + ".txt"):
        img = Image.open(prj_path + "/" + img_name + ".jpg")
        file_ori = open(prj_path + "/labels/" + img_name + ".txt", "r")
        lines_ori = file_ori.readlines()
        ref_path = src + "model/"
        if acc % 2 == 0:
            ref_path += "oiv7-zh.yaml"
        else:
            ref_path += "coco-zh.yaml"
        with open(ref_path, 'r', encoding='utf-8') as file_ref:
            lines_ref = file_ref.readlines()
        for line in lines_ori:
            line1 = line.split()
            num = int(line1[0])
            item.append(lines_ref[num].strip())
        file_ori.close()
        file_ref.close()
        
        uitem = list(set(item))
        
        wj = ["手枪", "炸弹", "凿子","匕首", "钻头", "叉子", "磨碎机", "厨房刀具", "刀", "导弹", "钉子", "步枪", "武器", "剪刀"]
        for it in range(len(uitem)):
            if uitem[it] in wj:
                uitem[it] += "【违禁品!】"
                wj_flag = True
        text = "共计" + str(len(uitem)) + "个物品"
        if wj_flag:
            text += "，其中包含违禁品！"
        else:
            text += "，未发现违禁品！"
        audio_filename="demo" + time.strftime("%Y%m%d%H%M%S", stamp) + ".mp3"
        synthesize_speech(text, audio_filename)
        
        return [img, "   ".join(uitem)]
    else:
        text = "未检测到物品！"
        audio_filename="demo" + time.strftime("%Y%m%d%H%M%S", stamp) + ".mp3"
        synthesize_speech(text, audio_filename)
        
        return [img, ""]

iface = gr.Interface(
    fn=calculate,
    inputs=[
        gr.Image(type="pil",label="原图"),
        gr.Slider(minimum=1, maximum=10, step=1, value=10, label="精度"),
    ],
    outputs=[
        gr.Image(type="pil", label="结果"),
        gr.Textbox(label="物品"),
    ],
)

iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7867
* Running on public URL: https://ac1dfc534fb9f3fac3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



image 1/1 C:\Users\26073\Workspace\detector\demo20250119182335.jpg: 640x640 4 Bottles, 1 Coffee cup, 1 Handgun, 1 Mug, 1845.8ms
Speed: 11.0ms preprocess, 1845.8ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to C:\Users\26073\Workspace\detector\_demo20250119182335
1 label saved to C:\Users\26073\Workspace\detector\_demo20250119182335\labels
语音合成成功，音频保存为 C:\Users\26073\Workspace\detector\demo20250119182335.mp3

image 1/1 C:\Users\26073\Workspace\detector\demo20250119182353.jpg: 640x640 1 handbag, 4 bottles, 1 cup, 2 books, 1 scissors, 1877.0ms
Speed: 12.0ms preprocess, 1877.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to C:\Users\26073\Workspace\detector\_demo20250119182353
1 label saved to C:\Users\26073\Workspace\detector\_demo20250119182353\labels
语音合成成功，音频保存为 C:\Users\26073\Workspace\detector\demo20250119182353.mp3

image 1/1 C:\Users\26073\Workspace\detector\demo20250119182408.jpg: 640x640 4 Bottles, 1 Coffee 